In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [ ]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
import numpy as np
from qiskit import ClassicalRegister
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.
# This notebook is for a simulation of the protocol without an attacker.


In [ ]:
def create_entangled_pair():
    qc = QuantumCircuit(2)
    qc.x(1)
    qc.h(0)
    qc.cx(0, 1)
    qc.z(0)
    return qc

def apply_measurement(qc, qubit, basis):
    if basis == 1:
        qc.h(qubit)
    elif basis == 2:
        qc.sdg(qubit)
        qc.h(qubit)
    elif basis == 3:
        qc.s(qubit)
        qc.h(qubit)

def quantum_choice():
    qc = QuantumCircuit(1, 1)
    theta = 2 * np.arccos(np.sqrt(1/3))
    qc.ry(theta, 0)
    qc.measure(0, 0)
    result = AerSimulator().run(qc, shots=1).result()
    if '0' in result.get_counts():
        return 1
    else:
        qc = QuantumCircuit(1, 1)
        qc.h(0)
        qc.measure(0, 0)
        result = AerSimulator().run(qc, shots=1).result()
        return 2 if '0' in result.get_counts() else 3

def simulate_ekert91(num_pairs):
    alice_bases, bob_bases, results = [], [], []
    simulator = AerSimulator()
    for _ in range(num_pairs):
        qc = create_entangled_pair()
        a_basis = quantum_choice()
        b_basis = quantum_choice()
        apply_measurement(qc, 0, a_basis)
        apply_measurement(qc, 1, b_basis)
        qc.measure_all()
        result = simulator.run(qc, shots=1).result()
        outcome = list(result.get_counts().keys())[0]
        alice_bases.append(a_basis)
        bob_bases.append(b_basis)
        results.append((int(outcome[0]), int(outcome[1])))
    return alice_bases, bob_bases, results
